# hCassandra runTests

## The Basics

### Objective

   This test aims to **automate**:
   
   (1) The execution of the Hydra Cassandra Stress Test (hCassandra) for increasing client load.
   (2) The generation of a full-report containing performance results in the form of tables and graphs. 
   
   To this end, tests are run for different client loads (defined by the user), i.e., increasing the number of clients writing and reading to the database.

### Customizing the Test

   Modify **total_num_clients** to change the number of clients per run and **total_ops_count** for modifyng the total number of operations per run.
   
   Current tests have been run for a maximum of **X** clients, a fixed number of 1,000,000 operations against a 3-node Cassandra Cluster (for further details on Software & Hardware specs please refer to the *Test Results/Specs* section.
   
### Useful HINTS for running the test

- If test has been previously executed and output is still shown, you can restart (delete former results) by selecting in the top menu Cell -> All Output -> Clear
- To run test, step on top of the code cells and press the 'run cell' button on the top menu.
- When RUN is finished, generate your own report by selecting FILE -> Download as -> Markdown (.md) (or any other preferred format.

---

## hCassandra Test 1: Fixed Number of Stress Clients (Debug Mode) 

---

The following test runs a SINGLE execution of the Cassandra Test for a fixed number of clients and operations. Runs in debug mode: showing logger info during execution. 

In [ ]:
!python ./src/hCassandra_test.py --cluster_ips='10.10.3.20,10.10.3.113,10.10.3.119' --total_client_count=10 --total_ops_count=100000

---

## hCassandra Test 2: Increasing the Number of stress clients (multiple runs)

---


**IMPORTANT**:

   If you want to change the number of clients and/or number of operations for your test, please set values to desired in the following section:


In [35]:
# Define num Client(s) / Operation(s) 
# total_num_clients = [10, 20, 100, 500, 1000]
total_num_clients = [10, 50, 100, 500, 1000]
total_ops_count = [1000000]
cassandra_cluster_ips = '10.10.3.20,10.10.3.113,10.10.3.119'

Next functions are some 'util' functions aimed to ease result processing. 

In [31]:
import json
import ast

def get_result(test_stdout):
    """This Function gets (filters) the Cassandra Test Results from stdout"""
    index_start = test_stdout.find('Cassandra Stress Results: \n')
    index_end = test_stdout.find('Calling Server shutdown')
    if index_start != -1:
        results = test_stdout[(index_start + len('Cassandra Stress Results: \n')):index_end]
        res_dict = ast.literal_eval(results)
        return res_dict
    else:
        return {}

The following block of code is the actual **EXECUTION OF THE CASSANDRA SCALE TESTS**. This may take a couple of minutes:

In [ ]:
import subprocess
import os
import json

hCassandra_results = dict()

print 'STARTING CASSANDRA STRESS TESTS \n'
# Execute hCassandra_test for given client_count
for idx1, clients in enumerate(total_num_clients):
    for idx2, ops in enumerate(total_ops_count):
        print ('Test (%s/%s) in progress.. Please wait until test is completed..' % ((len(total_ops_count) * idx1) + idx2 + 1,len(total_num_clients) * len(total_ops_count)))
        # Execute hCassandra_test.py (python script for hCassandra Scale Test)
        hcass_cmd = "python ./src/hCassandra_test.py --cluster_ips=%s --total_client_count=%s --total_ops_count=%s" % (cassandra_cluster_ips, clients, ops)
        stress_test = subprocess.Popen(hcass_cmd, stdout=subprocess.PIPE,
                                          stderr=subprocess.PIPE, shell=True, preexec_fn=os.setsid)
        stdout, stderr = stress_test.communicate()
        results_dict = get_result(stdout)
        if len(results_dict) <= 1:
            print ('There was an ERROR while attempting to parse stdout...')
            print 'STDOUT: %s' % stdout
            print 'STDERR: %s' % stderr
        if not str(clients) in hCassandra_results:
            hCassandra_results[str(clients)] = dict()
        hCassandra_results[str(clients)][str(ops)] = results_dict
        print 'Test SUCCESFULLY completed... \n'

print 'END OF TESTS:'
print 'ALL TESTS HAVE BEEN COMPLETED. PLEASE PROCEED TO GENERATE GRAPHS & TABLES WITH PERFORMANCE RESULTS.'

   ---
   
   Wait until RESULTS (**hCassandra_results**) are generated for all cases, and then execute the following blocks to generate (1) Tables with results (markdown compatible) and (2) Graphs
   
   The **END OF TEST** is indicated by a message. Please wait...
   
   ---

### RESULT PROCESSING & TABLE/ GRAPH GENERATION

Next, we reorder results per # of operations. In other words, for a fixed number of operations we show how performance changes when the number of clients increases. 

In [33]:
class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of 
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in 
        IPython Notebook. """
    
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            
            for col in row:
                html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)

In [36]:
results_per_ops = dict()

# Table Format
header = [
            '# ops',
            '# Clients',
            'total_ops',
            'op/s',
            'pk/s',
            'med',
            '.95',
            '.99',
            'max',
            'max_ms',
            'sdv_ms',
            'op_time'
        ]

data_matrix_write = ListTable()
data_matrix_read = ListTable()

data_matrix_write.append(header)
data_matrix_read.append(header)

for ops in total_ops_count:
    results_per_ops[str(ops)] = dict()
    first = True
    for clients in total_num_clients:
        if str(clients) in hCassandra_results:
            res_dict = ast.literal_eval(hCassandra_results[str(clients)][str(ops)])
            results_per_ops[str(ops)][str(clients)] = res_dict
            ops_fixed = ""
            if first:
                ops_fixed = ops
            data_matrix_write.append([ops_fixed, clients, res_dict['write']['total ops'], res_dict['write']['op/s'], res_dict['write']['pk/s'], res_dict['write']['med'], res_dict['write']['.95'], res_dict['write']['.99'], res_dict['write']['max'], res_dict['write']['max_ms'], res_dict['write']['sdv_ms'], res_dict['write']['op_time']])
            data_matrix_read.append([ops_fixed, clients, res_dict['read']['total ops'], res_dict['read']['op/s'], res_dict['read']['pk/s'], res_dict['read']['med'], res_dict['read']['.95'], res_dict['read']['.99'], res_dict['read']['max'], res_dict['read']['max_ms'], res_dict['read']['sdv_ms'], res_dict['read']['op_time']])
            first = False

### Result Generation: Table Format

Next, results are displayed in a Table, following the markdown format. Please, copy this table into your readme.results to keep track of results between test executions.  



The next table represents the results for the **WRITE** Operations:

---

*Table 1. "Cassandra Performance over WRITE Operation."*

In [37]:
data_matrix_write

# ops,# Clients,total_ops,op/s,pk/s,med,.95,.99,max,max_ms,sdv_ms,op_time
1000000,10,[1000000],[15639],[15639],[0.6],[0.8],[1.0],[125.2],[],[0.0],[' 00:01:03']
,50,"[1000000, 1000000, 1000000, 1000000, 1000000]","[8515, 8777, 7208, 7183, 8720]","[8515, 8777, 7208, 7183, 8720]","[0.8, 0.8, 0.9, 0.9, 0.8]","[2.2, 2.2, 2.8, 2.9, 2.1]","[4.0, 4.0, 5.7, 6.2, 3.9]","[212.1, 211.4, 211.1, 212.9, 219.0]",[],"[0.0, 0.0, 0.0, 0.0, 0.0]","[' 00:01:57', ' 00:01:53', ' 00:02:18', ' 00:02:19', ' 00:01:54']"
,100,"[1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000]","[5266, 5188, 5176, 5233, 5105, 5137, 5282, 5212, 5194, 5202]","[5266, 5188, 5176, 5233, 5105, 5137, 5282, 5212, 5194, 5202]","[1.1, 1.2, 1.2, 1.1, 1.2, 1.2, 1.1, 1.2, 1.2, 1.2]","[4.3, 4.3, 4.2, 4.4, 4.4, 4.3, 4.3, 4.3, 4.4, 4.2]","[8.2, 8.9, 8.2, 8.6, 8.4, 8.4, 8.2, 8.9, 9.0, 8.3]","[229.3, 229.1, 230.3, 228.4, 230.4, 233.6, 228.7, 229.2, 229.2, 235.6]",[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[' 00:03:09', ' 00:03:12', ' 00:03:13', ' 00:03:11', ' 00:03:15', ' 00:03:14', ' 00:03:09', ' 00:03:11', ' 00:03:12', ' 00:03:12']"
,500,"[1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 999994, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 999999, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000]","[1340, 1269, 1249, 1230, 1294, 1087, 1101, 1057, 1275, 1085, 1261, 1094, 1143, 1273, 1288, 1252, 1278, 1084, 1287, 1299, 1114, 1292, 1072, 1269, 1069, 1315, 1242, 1258, 1280, 1252, 1299, 1255, 1261, 1274, 1082, 1276, 1064, 1345, 1291, 1099, 1068, 1051, 1299, 1332, 1076, 1331, 1273, 1086, 1246, 1234]","[1340, 1269, 1249, 1230, 1294, 1087, 1101, 1057, 1275, 1085, 1261, 1094, 1143, 1273, 1288, 1252, 1278, 1084, 1287, 1299, 1114, 1292, 1072, 1269, 1069, 1315, 1242, 1258, 1280, 1252, 1299, 1255, 1261, 1274, 1082, 1276, 1064, 1345, 1291, 1099, 1068, 1051, 1299, 1332, 1076, 1331, 1273, 1086, 1246, 1234]","[2.4, 2.4, 2.5, 2.5, 2.2, 3.3, 3.2, 3.4, 2.3, 3.3, 2.4, 3.3, 3.1, 2.4, 2.4, 2.5, 2.3, 3.3, 2.3, 2.4, 3.3, 2.2, 3.3, 2.5, 3.3, 2.3, 2.5, 2.3, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 3.3, 2.3, 3.3, 2.2, 2.3, 3.1, 3.3, 3.3, 2.3, 2.4, 3.4, 2.2, 2.4, 3.2, 2.4, 2.5]","[24.3, 26.4, 27.7, 27.7, 26.0, 29.2, 29.1, 30.4, 26.5, 30.0, 26.3, 29.4, 27.9, 26.7, 26.0, 27.9, 27.0, 29.9, 26.6, 25.6, 29.0, 26.6, 30.8, 27.3, 29.6, 25.7, 27.4, 27.0, 26.5, 27.0, 26.8, 27.0, 27.1, 26.6, 30.3, 26.5, 30.1, 25.1, 26.6, 29.0, 30.0, 29.9, 26.3, 25.2, 30.0, 26.0, 27.0, 29.1, 26.7, 28.1]","[43.9, 46.7, 49.3, 49.0, 47.2, 50.9, 51.6, 52.4, 47.3, 51.6, 47.5, 51.7, 48.9, 46.6, 47.6, 49.2, 48.8, 52.2, 47.9, 46.1, 51.7, 48.4, 54.5, 49.4, 51.6, 46.2, 49.4, 48.0, 47.3, 49.8, 48.4, 48.5, 48.5, 48.6, 52.7, 48.5, 51.7, 46.2, 47.9, 50.4, 52.4, 52.2, 47.8, 47.6, 52.1, 46.7, 47.1, 50.4, 47.5, 50.5]","[271.8, 276.9, 271.0, 255.8, 270.3, 276.5, 269.8, 306.3, 300.6, 261.2, 275.4, 288.6, 279.2, 275.4, 271.1, 255.7, 270.2, 277.2, 272.4, 274.2, 261.2, 274.5, 302.5, 250.7, 303.4, 285.2, 276.4, 276.3, 303.1, 274.4, 269.6, 283.2, 302.6, 276.4, 289.3, 282.2, 290.0, 273.2, 260.1, 272.1, 303.1, 302.2, 250.7, 273.7, 270.6, 274.0, 276.3, 261.8, 272.6, 269.6]",[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[' 00:12:26', ' 00:13:08', ' 00:13:20', ' 00:13:32', ' 00:12:52', ' 00:15:20', ' 00:15:08', ' 00:15:45', ' 00:13:04', ' 00:15:21', ' 00:13:13', ' 00:15:14', ' 00:14:34', ' 00:13:05', ' 00:12:56', ' 00:13:18', ' 00:13:02', ' 00:15:22', ' 00:12:56', ' 00:12:49', ' 00:14:58', ' 00:12:54', ' 00:15:33', ' 00:13:07', ' 00:15:35',

The next table represents the results for the **READ** Operations:

---

*Table 1. "Cassandra Performance over READ Operation."*

In [38]:
data_matrix_read

# ops,# Clients,total_ops,op/s,pk/s,med,.95,.99,max,max_ms,sdv_ms,op_time
1000000,10,[1000000],[15883],[15883],[0.6],[0.8],[1.0],[46.6],[],[0.0],[' 00:01:02']
,50,"[1000000, 1000000, 1000000, 1000000, 1000000]","[7202, 7204, 7973, 7809, 7101]","[7202, 7204, 7973, 7809, 7101]","[0.9, 0.9, 0.9, 0.9, 1.0]","[2.9, 3.0, 2.6, 2.8, 3.0]","[4.9, 5.4, 4.9, 5.1, 5.3]","[63.4, 59.2, 56.0, 62.5, 59.0]",[],"[0.0, 0.0, 0.0, 0.0, 0.0]","[' 00:02:18', ' 00:02:18', ' 00:02:05', ' 00:02:08', ' 00:02:20']"
,100,"[1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000]","[4438, 4472, 4540, 4382, 4621, 4533, 4547, 4433, 4474, 4415]","[4438, 4472, 4540, 4382, 4621, 4533, 4547, 4433, 4474, 4415]","[1.3, 1.3, 1.3, 1.4, 1.2, 1.3, 1.3, 1.3, 1.3, 1.3]","[5.4, 5.5, 5.3, 5.5, 5.4, 5.4, 5.4, 5.3, 5.2, 5.3]","[11.7, 11.3, 11.2, 11.0, 10.9, 11.1, 11.2, 11.1, 10.6, 10.7]","[83.6, 91.7, 84.0, 79.5, 84.4, 81.3, 78.1, 87.8, 93.0, 84.5]",[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[' 00:03:45', ' 00:03:43', ' 00:03:40', ' 00:03:48', ' 00:03:36', ' 00:03:40', ' 00:03:39', ' 00:03:45', ' 00:03:43', ' 00:03:46']"
,500,"[1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000]","[971, 983, 993, 995, 956, 1081, 1080, 1107, 966, 1082, 976, 1078, 1049, 975, 963, 979, 974, 1081, 973, 955, 1059, 956, 1078, 976, 1093, 967, 999, 972, 988, 981, 971, 973, 990, 976, 1072, 956, 1100, 972, 959, 1072, 1110, 1116, 969, 954, 1086, 972, 973, 1076, 988, 1007]","[971, 983, 993, 995, 956, 1081, 1080, 1107, 966, 1082, 976, 1078, 1049, 975, 963, 979, 974, 1081, 973, 955, 1059, 956, 1078, 976, 1093, 967, 999, 972, 988, 981, 971, 973, 990, 976, 1072, 956, 1100, 972, 959, 1072, 1110, 1116, 969, 954, 1086, 972, 973, 1076, 988, 1007]","[2.4, 2.2, 2.0, 2.1, 2.6, 2.0, 2.2, 2.3, 2.4, 2.2, 2.3, 2.0, 1.9, 2.5, 2.4, 2.4, 2.4, 2.4, 2.3, 2.5, 2.2, 2.5, 2.3, 2.3, 2.3, 2.4, 2.0, 2.3, 2.1, 2.3, 2.3, 2.3, 2.0, 2.3, 2.1, 2.7, 2.4, 2.4, 2.6, 2.1, 2.3, 2.2, 2.3, 2.5, 2.2, 2.4, 2.3, 2.3, 2.1, 1.8]","[37.4, 37.5, 36.9, 36.3, 37.4, 35.0, 35.2, 34.5, 36.1, 35.1, 36.3, 35.2, 35.8, 36.6, 37.2, 36.7, 36.3, 34.8, 37.3, 36.8, 35.3, 37.1, 34.8, 35.5, 34.5, 37.0, 37.1, 37.1, 37.3, 36.8, 36.8, 37.1, 37.0, 36.5, 35.3, 36.5, 34.3, 36.4, 36.8, 35.2, 33.4, 34.5, 37.4, 37.1, 34.7, 36.9, 37.3, 34.7, 37.1, 36.4]","[58.8, 58.5, 58.0, 58.6, 59.4, 54.8, 56.0, 56.2, 57.0, 56.0, 58.1, 57.0, 56.8, 57.5, 58.0, 57.1, 57.5, 55.8, 58.7, 58.2, 57.5, 58.2, 56.0, 56.3, 55.8, 58.1, 58.2, 58.2, 58.4, 57.7, 57.6, 58.1, 58.2, 58.5, 56.3, 58.1, 55.8, 56.7, 58.4, 56.8, 55.1, 56.6, 59.2, 59.1, 56.5, 57.8, 58.4, 56.6, 59.0, 57.1]","[237.8, 191.3, 180.0, 180.8, 188.3, 184.2, 159.7, 207.1, 189.9, 436.2, 195.4, 199.9, 187.1, 190.8, 245.4, 196.1, 261.2, 176.6, 189.3, 245.1, 178.7, 181.3, 178.8, 176.3, 175.0, 310.0, 220.8, 193.9, 191.3, 182.8, 189.7, 191.2, 212.0, 188.8, 163.7, 240.2, 180.0, 182.1, 178.2, 210.2, 171.0, 232.0, 195.6, 179.8, 167.5, 183.8, 196.4, 212.0, 201.9, 167.7]",[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[' 00:17:09', ' 00:16:57', ' 00:16:47', ' 00:16:45', ' 00:17:26', ' 00:15:25', ' 00:15:26', ' 00:15:03', ' 00:17:14', ' 00:15:24', ' 00:17:04', ' 00:15:27', ' 00:15:53', ' 00:17:05', ' 00:17:18', ' 00:17:01', ' 00:17:07', ' 00:15:25', ' 00:17:07', ' 00:17:26', ' 00:15:43', ' 00:17:25', ' 00:15:27', ' 00:17:04', ' 00:15:14', ' 00:17:13', ' 00:16:40', ' 00:17:08', ' 00:16:51', ' 00:16:59', ' 

### Result Generation: Graphs

Next, results are displayed in Graphs. 

--- 

**IMPORTANT**

Please, MODIFY the graphs name in order to avoid OVERWRITING previous results.



---

In [39]:
ops_second_graph_filename = "hCassandra_ops_sec_002"
median_latency_graph_filename = "hCassandra_med_002"

In [40]:
import plotly.plotly as py
from plotly.graph_objs import *
import operator
import numpy

data_matrix = [['# ops', '# Clients', 'total_ops', 'op/s', 'pk/s', 'med', '.95', '.99', 'max', 'max_ms', 'sdv_ms', 'op_time']]


traces_plot1 = []
traces_plot2 = []

# For each trace = client count
for ops_count, tests_per_trace in results_per_ops.iteritems():
    
    total_ops = []
    op_s = []
    op_s_r = []
    pk_s = []
    med = []
    med_r = []
    p95 = []
    p99 = []
    max_lat = []
    max_ms = []
    sdv_ms = []
    op_time = []
    
    for num_clients, res_dict in tests_per_trace.iteritems():
        op_s.append(sum((ops for ops in res_dict['write']['op/s'])))
        med.append(numpy.median(res_dict['write']['med']))
        op_s_r.append(sum((ops for ops in res_dict['read']['op/s'])))
        med_r.append(numpy.median(res_dict['read']['med']))
        # Un-Comment for any other feature you want to depict...
        
        #total_ops.append(sum((ops for ops in res_dict['write']['total ops'])))
        #pk_s.append(res_dict['write']['pk/s'])
        #p95.append(res_dict['write']['.95'])
        #p99.append(res_dict['write']['.99'])
        #max_lat.append(res_dict['write']['max'])
        #max_ms.append(res_dict['write']['max_ms'])
        #sdv_ms.append(res_dict['write']['sdv_ms'])
        #op_time.append(res_dict['write']['op_time'])
        
        
    trace_plot1 = Scatter(
          x=total_num_clients,
          y=op_s, 
          mode = 'lines+markers',
          name = 'ops_count = ' + str(ops_count) + '[WRITE]',
          line=dict(
            shape='spline'
            )
        )
    
    trace_plot2 = Scatter(
          x=total_num_clients,
          y=op_s_r, 
          mode = 'lines+markers',
          name = 'ops_count = ' + str(ops_count) + '[READ]',
          line=dict(
            shape='spline'
            )
        )
        
    trace_plot3 = Scatter(
          x=total_num_clients,
          y=med, 
          mode = 'lines+markers',
          name = 'ops_count = ' + str(ops_count) + '[WRITE]', 
          line=dict(
            shape='spline'
            )
        )

    trace_plot4 = Scatter(
          x=total_num_clients,
          y=med_r, 
          mode = 'lines+markers',
          name = 'ops_count = ' + str(ops_count) + '[READ]', 
          line=dict(
            shape='spline'
            )
        )
    
    traces_plot1.append(trace_plot1)
    traces_plot1.append(trace_plot2)
    traces_plot2.append(trace_plot3)
    traces_plot2.append(trace_plot4)


### Result Generation: operations per second vs. client count

The following graph illustrates how, the number of operations per second changes while the number of clients increases 

In [41]:
data = Data(traces_plot1)
# Edit the layout
layout = dict(title = 'op/s vs. # Clients',
              xaxis = dict(title = '# clients'),
              yaxis = dict(title = 'op/s'),
              )

# Plot and embed in notebook
fig = dict(data=data, layout=layout)
py.iplot(fig, filename = ops_second_graph_filename)

/home/annyz/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.



### Result Generation: median latency vs. client count

The following graph illustrates median latency in miliseconds for each operation during that run as the number of clients increases. 

In [42]:
data = Data(traces_plot2)
# Edit the layout
layout = dict(title = 'Median Latency vs. Client Count',
              xaxis = dict(title = '# Clients'),
              yaxis = dict(title = 'Median Latency [ms]'),
              )

# Plot and embed in notebook
fig = dict(data=data, layout=layout)
py.iplot(fig, filename = median_latency_graph_filename)

/home/annyz/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.

